In [1]:
!pip install azureml-train-automl-client, azureml-train-automl-runtime, azureml-dataset-runtime, azureml-automl-runtime, azureml-automl-core
!pip show azureml-train-automl
!pip install azureml-widgets
!pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

ERROR: Invalid requirement: 'azureml-train-automl-client,'
Name: azureml-train-automl
Version: 1.26.0
Summary: automl
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: azureml-train-automl-runtime, azureml-train-automl-client, azureml-dataset-runtime, azureml-automl-runtime, azureml-automl-core
Required-by: azureml-automl-dnn-nlp


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))


Ready to use Azure ML 1.26.0 to work with 429test


In [3]:
from azureml.core import Experiment
from azureml.core import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import joblib
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ws.get_default_datastore()
dataset = Dataset.get_by_name(ws, name='zombies')
zombies_train, zombies_test = dataset.random_split(percentage=0.7, seed=123) 
# zombies = dataset.to_pandas_dataframe()

In [4]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('regression'):
    print(metric)

normalized_mean_absolute_error
spearman_correlation
r2_score
normalized_root_mean_squared_error


In [5]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'r2_score',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [6]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',

                             debug_log='automated_ml_errors.log',
                             training_data=zombies_train,
                             label_column_name="Bounty",
                             **automl_settings)

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [7]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "zombie_aml")
local_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
zombie_aml,AutoML_2d992e21-4918-4af7-bb78-1c0fe866c125,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed